# DQMC (Determinant Quantum Monte-Carlo) Method

## About the notebook

This notebook **may not** complete. Last update time: 2020-10-13.

Because it is hard to say the author understands DQMC, this notebook may contain errors and bugs. If you found one, feel free to [send an email](mailto:chli@iphy.ac.cn) to tell me. I'm willing to learn from you.

This Jupyter Notebook occurs as a supplementary to [a great note on the DQMC](https://quantummc.xyz/2020/10/11/dqmc-note/) written by my friend(referred to The Note below). And this Jupyter notebook will focus on the implement of the DQMC. 

```
    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.
```

## Prerequisites

As [mentioned](https://quantummc.xyz/dqmc-note/part-1-prerequisites/) in The Note. You should have some basic knowledge about Monte-Carlo method, Statistical Physics and Quantum Mechanics.

## The Hubbard Model

Here is the Hamiltonian of Hubbard model, if we ignore the chemical potential part.
$$
\begin{equation}
H = -t\sum_{<ij>,\sigma}(c^\dagger_{i\sigma} c_{j\sigma} + h.c.) + U\sum_{i}(n_{i\uparrow}-\frac{1}{2}) (n_{i\downarrow}-\frac{1}{2})
\end{equation}
$$

And clearly it could be separated to two parts: kinetic and potiential, which are

$$
\begin{equation}
\begin{aligned}
H_T = -t\sum_{<ij>,\sigma}(c^\dagger_{i\sigma} c_{j\sigma} + h.c.) \\
H_V = U\sum_{i}(n_{i\uparrow}-\frac{1}{2}) (n_{i\downarrow}-\frac{1}{2})
\end{aligned}
\end{equation}
$$

In order to apply the Monte-Carlo method, we should write down the partition function
$$
\begin{equation}
    \DeclareMathOperator{\Tr}{Tr}
    Z = \Tr\left[e^{\beta H}\right] \\
      = \Tr\left[e^{-\varDelta\tau \cdot M \cdot H}\right] \\
      = \Tr\left[(e^{-\varDelta\tau \cdot H})^M\right]
\end{equation}
$$
and seperate $\beta$ into $M$ slices, so $\beta = -\varDelta\tau \cdot M $. 

We can further apply Suzuki-Trotter decomposition on the term $ e^{-\varDelta\tau \cdot H} $,

$$
\begin{equation}
    e^{-\varDelta\tau H} = e^{-\varDelta\tau H_V}e^{-\varDelta\tau H_T} + O[(\varDelta\tau)^2]
\end{equation}
$$

where $H_T = -t\sum_{<ij>,\sigma}(c^\dagger_{i\sigma} c_{j\sigma} + h.c.)$, $H_V = U\sum_i(n_{i\uparrow}-\frac{1}{2}) (n_{i\downarrow}-\frac{1}{2})$.

### The potiential part $H_V$

Since $n_{i\sigma} = c_{i\sigma}^\dagger c_{i\sigma}$ , we then have a four fermion operator production in the $H_V$. This could be solved by apply Hubbard-Stratonovich transformation. The details and the proof could be found in [2nd part of The Note](https://quantummc.xyz/dqmc-note/part-2-starting-from-hubbard-model/). As a result, for a single site, we have

$$
\begin{equation}
e^{-\varDelta\tau U (n_{\uparrow}-\frac{1}{2}) (n_{\downarrow}-\frac{1}{2})} = \frac{1}{2} e^{-\varDelta\tau \frac{|U|}{4}} \sum_{\tau} e^{\alpha s_\tau (n_{\uparrow}-n_{\downarrow})}
\end{equation}
$$
where $\tau$ marks imaginary time index, $\cosh \alpha = e^{\varDelta\tau \frac{|U|}{2}}$, $\frac{1}{2} e^{-\varDelta\tau \frac{|U|}{4}}$ will then be denoted as $\lambda$. And here we have introduced the auxiliary field $s_\tau$.

Next we can write the $e^{-\varDelta \tau H_V}$ could be written as

$$
\begin{equation}
    e^{-\varDelta\tau H_V} = e^{\varDelta\tau U\sum_{i}(n_{i\uparrow}-\frac{1}{2}) (n_{i\downarrow}-\frac{1}{2})} = \prod_{i} \lambda \sum_{s_{i,\tau}=\pm 1} e^{\alpha s_{i,\tau}(n_{i,\uparrow} - n_{i,\downarrow})}\\
    = \lambda^{N} \sum_{s_{i,\tau}=\pm 1} ( \prod_{i} e^{\alpha s_{i,\tau}n_{i,\uparrow}} \prod_{i} e^{-\alpha s_{i,\tau} n_{i,\downarrow}})
\end{equation}
$$

### The kinetic part $H_T$

The kinetic term in Hamiltonian with $T$ could be written as
$$
e^{-\varDelta\tau H_T} = e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\uparrow} c_{j\uparrow} + c^{\dagger}_{j\uparrow}c_{i\uparrow})} e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\downarrow} c_{j\downarrow} + c^{\dagger}_{j\downarrow}c_{i\downarrow})} = \prod_{\sigma} e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\sigma} c_{j\sigma} + c^{\dagger}_{j\sigma}c_{i\sigma})}
$$

For Hubbard model, the $<i,j>$ denotes for the nearest neighbor of 2 lattice site, and $\sigma = \uparrow, \downarrow$ here. Express above in matrix, we could have

$$
-t \sum_{<ij>} (c^\dagger_{i\uparrow} c_{j\uparrow} + c^{\dagger}_{j\uparrow}c_{i\uparrow}) = \boldsymbol{c}_\uparrow^\dagger T \boldsymbol{c}_\uparrow
$$

then

$$
e^{\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\uparrow} c_{j\uparrow} + c^{\dagger}_{j\uparrow}c_{i\uparrow})} = e^{ -\varDelta\tau \boldsymbol{c}_\uparrow^\dagger T \boldsymbol{c}_\uparrow}
$$

and

$$
\begin{equation}
    e^{-\varDelta\tau H_T} = e^{ -\varDelta\tau \boldsymbol{c}_\uparrow^\dagger T \boldsymbol{c}_\uparrow} e^{ -\varDelta\tau \boldsymbol{c}_\downarrow^\dagger T \boldsymbol{c}_\downarrow}
\end{equation}
$$

### The determinant

We have successfully transform the original Hamiltonian into a obscure form. But why? The answer is connect to the key part of DQMC, determinant. When we calculate the trace of quadratic form of fermion operator on the exponent like $e^{c^\dagger A c}$, we could always convert it into a determinant. That is to say

$$
\begin{equation}
\operatorname{Tr}\left[e^{-\sum_{i, j} \hat{c}_{i}^{\dagger} A_{i, j} \hat{c}_{j}}\right]=\operatorname{Det}\left[\mathbf{1}+e^{-\mathbf{A}}\right].
\end{equation}
$$

The proof and generalization could be found at [2nd part of The Note](https://quantummc.xyz/dqmc-note/part-2-starting-from-hubbard-model/). That's why we perform HS transformation on the $H_V$. The trace in the partition function could then be written as a series of determinant.

$$
\begin{equation}\operatorname{Tr}\left[e^{-\sum_{i, j} c_{i}^{\dagger} A_{i, j} c_{j}} e^{-\sum_{i, j} c_{i}^{\dagger} B_{i, j} c_{j}}\right]=\operatorname{Det}\left[\mathbf{1}+e^{-\mathbf{A}} e^{-\mathbf{B}}\right]\end{equation}
$$

The $\mathbf{A}$ and the $\mathbf{B}$ here could be regarded as $\mathbf{T}$ and the $\mathbf{V}$ in the Hamiltonian.

Finally the partition function is written as

$$
\begin{equation}
\begin{aligned}
    Z &= \Tr\left[(e^{-\varDelta\tau \cdot H})^M\right] \\
    &= \Tr\left[(e^{-\varDelta\tau \cdot (H_T + H_V)})^M\right] \\
    &= \Tr\left[\prod_{\tau=1}^M (e^{-\varDelta\tau H_T} e^{-\varDelta\tau H_V}  )\right] \\
    &= \Tr\left[\prod_{\tau=1}^M \left(e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\uparrow} c_{j\uparrow} + c^{\dagger}_{j\uparrow}c_{i\uparrow})} e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\downarrow} c_{j\downarrow} + c^{\dagger}_{j\downarrow}c_{i\downarrow})}  \lambda^{N} \sum_{s_{i,\tau}=\pm 1} ( e^{\alpha \sum_i s_{i,\tau}n_{i,\uparrow}} e^{-\alpha  \sum_i s_{i,\tau} n_{i,\downarrow}}) \right)\right] \\
    &= \lambda^{NM} \Tr \left[\prod_{\tau=1}^M \left(  \sum_{s_{i,\tau}=\pm 1}( e^{\alpha \sum_i s_{i,\tau}n_{i,\uparrow}} e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\uparrow} c_{j\uparrow} + c^{\dagger}_{j\uparrow}c_{i\uparrow})} ) ( e^{-\alpha  \sum_i s_{i,\tau} n_{i,\downarrow}} e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\downarrow} c_{j\downarrow} + c^{\dagger}_{j\downarrow}c_{i\downarrow})} ) \right)\right] \\
    &= \lambda^{NM} \Tr \left[\prod_{\tau=1}^M \left(  \sum_{s_{i,\tau}=\pm 1} \prod_\sigma ( e^{\alpha \sum_i s_{i,\tau}n_{i,\sigma}} e^{-\varDelta\tau t \sum_{<ij>} (c^\dagger_{i\sigma} c_{j\sigma} + c^{\dagger}_{j\sigma}c_{i\sigma})}  ) \right)\right] \\
    &= \lambda^{NM} \sum_{s_{i, \tau}} \prod_{\sigma} \det \left[ I + B_{M}^{\sigma}B_{M-1}^{\sigma}B_{M-2}^{\sigma}\cdots B_{2}^{\sigma}B_{1}^{\sigma} \right]
\end{aligned}
\end{equation}
$$

where we define

$$
B_\tau^\sigma = B^{\sigma}(\tau \varDelta\tau,(\tau-1) \varDelta\tau) = e^{\alpha S_1} e^{-\varDelta\tau T}
$$

or we could write

$$
\begin{equation}
    B^{\sigma}(l_2  \varDelta\tau, l_1  \varDelta\tau) = \prod_{l = \tau_1+1}^{\tau_2} B_{l}^\sigma,\tau_2>\tau_1
\end{equation}
$$

In the above equation, the $s_i$ is the lattice site index, $\tau$ or $l$ is the imaginary time index (from $1$ to $M$), $\sigma$ is the spin orientation index($\uparrow, \downarrow$).

### The Green's Function


Equal time Green's function is defined

$$
G_{ij}(\tau,\tau) = \left\langle c_{i}(\tau) c_{j}^{\dagger}(\tau)\right\rangle
$$

For an observable, we have 

$$
\begin{equation}\langle \hat{O}(\tau)\rangle=\frac{\operatorname{Tr}\left[e^{-\beta H} \hat{O}(\tau)\right]}{\operatorname{Tr}\left[e^{-\beta H}\right]}=\sum_{C} \mathrm{P}_{C}\langle \hat{O}(\tau)\rangle_{C}+O\left(\Delta_{\tau}^{2}\right)\end{equation}
$$

and 

$$
\langle\hat{O}\rangle_{\mathcal{C}} = \operatorname{Tr}\left[\left(1-\left(1+B_{\mathcal{C}}(\tau, 0) B_{\mathcal{C}}(\beta, \tau)\right)^{-1}\right) O\right]
$$

then we have

$$
\begin{equation}
G(\tau, \tau)=[\mathbf{1}+B(\tau, 0) B(\beta, \tau)]^{-1} 
\end{equation}
$$

the details is mentioned at [3rd part of The Note](https://quantummc.xyz/dqmc-note/part-3-measurement-and-update/).

## Monte-Carlo Update



The weight to a specific configuration
$$
\begin{equation}W_{C}=\prod_{\sigma=\uparrow, \downarrow} \operatorname{det}\left[\mathbf{I}+ \mathbf{B}_{C}^{\sigma}(\beta, 0)\right]\end{equation}
$$

after update the $l$-th auxiliary field at site $i$, denote the updated configure as $S'_l$(original is $S_l$), the updated configuration weight is

$$

\begin{equation}
\begin{aligned}

W_{C}'=& \operatorname{det}[\mathbf{I}+\mathbf{B}(\beta, \tau) e^{V\left(\vec{S}^{\prime}_{l}\right)} e^{-\Delta \tau T} \mathbf{B}(\tau -\Delta \tau, 0)]\\

=& \operatorname{det}[\mathbf{I}+\mathbf{B}(\beta, \tau) e^{V\left(\vec{S}^{\prime}_{l}\right)} \underbrace{e^{-V\left(\vec{S}_{l}\right)} e^{V\left(\vec{S}_{l}\right)} }_{I} e^{-\Delta \tau T} \mathbf{B}(\tau -\Delta \tau, 0)] \\

=&  \operatorname{det}[\mathbf{I}+\mathbf{B}(\beta, \tau) e^{V\left(\vec{S}^{\prime}_{l}\right)} e^{-V\left(\vec{S}_{l}\right)} \mathbf{B}(\tau , 0)]\\

=&\operatorname{det}[\mathbf{I}+\mathbf{B}(\beta, \tau) (\mathbf{I}+\mathbf{\Delta}) \mathbf{B}(\tau , 0)]

\end{aligned}
\end{equation}
$$

where $\mathbf{\Delta}=e^{V\left(\vec{S}^{\prime}_{l}\right)} e^{-V\left(\vec{S}_{l}\right)}-\mathbf{I}$.

The acceptance ratio is the ratio of weight

$$
\begin{equation}
R^\sigma = \operatorname{det}[\mathbf{I}+\mathbf{\Delta}(\mathbf{I}-\mathbf{G}^\sigma(\tau, \tau))]
\end{equation}
$$

Due to there are only one elements changed, the equation above could be simplified as 

$$
\begin{equation}
R^\sigma = 1+\mathbf{\Delta}^\sigma_{i i}\left(1-\mathbf{G}^\sigma_{i i}\right)
\end{equation}
$$

The equal time Green's Funtion is also updated with 

$$
\begin{equation}
\mathbf{G}^{\sigma\prime}(\tau, \tau) = \mathbf{G}^\sigma(\tau, \tau)[\mathbf{I}+\boldsymbol{\Delta}^\sigma(\mathbf{I}-\mathbf{G}^\sigma(\tau, \tau))]^{-1} \\
 = \mathbf{G}^\sigma(\tau, \tau) - \frac{1}{R^\sigma} \mathbf{G}^\sigma(\tau, \tau) \boldsymbol{\Delta}^\sigma (\mathbf{I} - \mathbf{G}^\sigma(\tau, \tau))
\end{equation}
$$
equation above could also be simplified. Details in [3rd part of The Note](https://quantummc.xyz/dqmc-note/part-3-measurement-and-update/).

After get equal time Green's Function at $\tau$, we can the calculate the Green's Function at $\tau \pm \varDelta\tau$ with

$$
\begin{equation}
\begin{aligned}
\mathbf{G}(\tau+\Delta \tau, \tau+\Delta \tau) &= \mathbf{B}(\tau+\Delta \tau, \tau) \mathbf{G}(\tau, \tau) \mathbf{B}^{-1}(\tau+\Delta \tau, \tau) \\
\mathbf{G}(\tau-\Delta \tau, \tau-\Delta \tau) &= \mathbf{B}^{-1}(\tau, \tau-\Delta \tau) \mathbf{G}(\tau, \tau) \mathbf{B}(\tau, \tau -\Delta \tau)
\end{aligned}
\end{equation}
$$

## Numerical Stability

It seems simple to calculate Green's Function at first glance. You just calculate it as mentioned above, first $B(\tau,0)B(\beta,\tau)$, then calculate inverse.  
However after doing this for a large $\beta$, the acquired Green's function will be inaccurate. 

Some techniqes could be apply to mitigate this problem. The key point is apply the SVD decomposition to the $B(\tau,0)$ and $B(\beta,\tau)$ matrix. And then calculate the green function with those $U$, $D$ and $V$s. For example,

$$
\begin{equation}
B_{C}\left(2 \tau_{1}, \tau_{1}\right) \underbrace{B_{C}\left(\tau_{1}, 0\right)}_{U_{1} D_{1} V_{1}}=\underbrace{\left(\left(B_{C}\left(2 \tau_{1}, \tau_{1}\right) U_{1}\right) D_{1}\right)}_{U_{2} D_{2} V} V_{1}=U_{2} D_{2} V_{2}
\end{equation}
$$

Then the equal time Green's Function could be get with
$$
\begin{equation}
\begin{aligned}

G(\tau, \tau) &=[1+B(\tau, 0) B(\beta, \tau)]^{-1} \\

&=\left[1+U_{R} D_{R} V_{R} V_{L} D_{L} U_{L}\right]^{-1} \\

&=U_{L}^{-1}\left[\left(U_{L} U_{R}\right)^{-1}+D_{R}\left(V_{R} V_{L}\right) D_{L}\right]^{-1} U_{R}^{-1} \\

&=U_{L}^{-1}\left[\left(U_{L} U_{R}\right)^{-1}+D_{R}^{\max } D_{R}^{\min }\left(V_{R} V_{L}\right) D_{L}^{\min } D_{L}^{\max }\right]^{-1} U_{R}^{-1} \\

&=U_{L}^{-1}\left(D_{L}^{\max }\right)^{-1}\left[\left(D_{R}^{\max }\right)^{-1}\left(U_{L} U_{R}\right)^{-1}\left(D_{L}^{\max }\right)^{-1}+D_{R}^{\min } V_{R} V_{L} D_{L}^{\min }\right]^{-1}\left(D_{R}^{\max }\right)^{-1} U_{R}^{-1}

\end{aligned}
\end{equation}
$$

## Implement

As you might or might not notice, this Jupyter notebook is written in Julia. Julia is a high performance laguage targeting scientific calculation. In order to run, it is required to install a Julia environment. Check `README.md` to get more information.

First, we need to define our model. As an example, we calculate the Hubbard Model on square lattice.

First, we should setting the parameter that used in simulation, and generate square lattice list, for every time slice we need to generate a 2-d lattice with each site in the lattice is set to either -1 or 1.

In [ ]:
include("hubbard.jl")
include("svd.jl")

Calculate imaginary time propagators $B^{\sigma}(\tau_2, \tau_1)$ matrix. $B^{\sigma}(\tau_2, \tau_1)$ is calculate with
$$
\begin{equation}
    B^{\sigma}(\tau_2,\tau_1) = B(\tau_2)\cdots B(\tau_1)
\end{equation}
$$

Calculate the Green function, defined as $G^{\sigma}(\tau,\tau) = (I + B^{\sigma}(\tau,0)B^{\sigma}(\beta,\tau))^{-1}$, $\tau$ indicates the imaginary time slice index.

We choose to use SVD decomposition to calculate the Green function. The reason is that when the $U$ is large, the matrix $I + B^{\sigma}(\tau,0)B^{\sigma}(\beta,\tau)$ will be more similar to the singular matrix, which causes the inaccuracy of the matrix inverse. In order to deal with the problem, we use SVD decomposition to separate the big and small part of the eigen value. 

$$
\begin{equation}
    G = (I + B(\tau,0)B(\beta,\tau))^{-1}\\
    = (I + U_R D_R V_R V_L D_L U_L)^{-1} \\
    = U_{L}^{-1} (D_{L}^{max})^{-1} [(D_{R}^{max})^{-1}(U_R U_L)^{-1} (D_{L}^{max})^{-1} + D_R^{min}V_R V_L D_{L}^{min}]^{-1} (D_{R}^{max})^{-1} U_R^{-1}
\end{equation}
$$

$D_{R}^{max}$ is a diagonal matrix derived from $D_{R}$, with the number **smaller** than 1 replaces by 1. $D_{R}^{min}$ is a diagonal matrix derived from $D_{R}$, with the number **larger** than 1 replaces by 1.

Calculate the update ratio

$$
\begin{equation}
    R = \prod_{\sigma} \det \left[ I + \Delta^{\sigma}(i,r) G^{\sigma}(\tau,\tau) \right] \\
     = \prod_{\sigma} (I + \Delta^{\sigma}_{ii}(i,\tau)(I - G_{ii}^{\sigma}(\tau,\tau)))
\end{equation}
$$

$i$ is the lattice list linear index, $\tau$ is imaginary time index, $\sigma$ is spin orientation index. The $\Delta$ is given by the below formula

$$
   \begin{equation}
      \Delta = e^{\alpha  \mathrm{Diag}[S']} e^{-\alpha \mathrm{Diag}[S]} - I
   \end{equation}
$$

## Monte-Carlo Sampling

The key idea behind the Monte-Carlo Sampling is Markov Chain Monte-Carlo. In the sampling space, we randomly choose a star point, and let the system evolve, this is mapped to a movement in the sampling space. After some time, the system will finally go to a state that we called equivalent state. 

In order to fulfill the detailed equivalence condition, we should give the movement in the sampling space an acceptation probability, to determine whether to move or not. 

### Sweep

We iterate through the imaginary time slices, and in each time slices, we also iterate the site on the lattice. According to the spin on each site, we can the calculate the acceptation ratio. This ratio will compare to a random generated number to determine whether we update the Green function or not. 

A full process will last from $0$ to $\beta$, and then $\beta$ to $0$. This whole process was called a sweep.

The Green function is updated with

$$
\begin{equation}
    G^{\sigma}(\tau,\tau)' = G^{\sigma}(\tau,\tau) - \frac{1}{R^{\sigma}} \Delta^{\sigma}(i,\tau) (I-G^{\sigma}(\tau,\tau))
\end{equation}
$$

In theory we should calculate equal time Green function at each imaginary time slice with

$$
\begin{equation}
    G^{\sigma}(\tau,\tau) = (I + B^{\sigma}(\tau, 0)) B^{\sigma}(\beta,\tau))^{-1}
\end{equation}
$$

However, this will cause huge number of calculation and make the process slow. So, we choose to calculate an approximate equal time Green function with B propagator, and thus

$$
\begin{equation}
    G^{\sigma}(\tau+1,\tau+1) = B^{\sigma}(\tau+1,\tau)G^{\sigma}(\tau,\tau)B^{\sigma}(\tau+1,\tau)^{-1} \\
    G^{\sigma}(\tau-1,\tau-1) = B^{\sigma}(\tau,\tau-1)^{-1}G^{\sigma}(\tau,\tau)B^{\sigma}(\tau,\tau-1)
\end{equation}
$$

As the imaginary time increase, the accuracy of the formula is decrease. So, we should recalculate Green function with the accurate formula above.

## Observable

The system kinetic energy is given by

$$
\begin{equation}
    T = - t \langle c_i^\dagger c_j \rangle
\end{equation}
$$

Double occupancy

$$
\begin{equation}
    D = \langle n_{i\uparrow} n_{j\downarrow} \rangle
\end{equation}
$$

Structure factor

$$
\begin{equation}
    S(Q) = \frac{1}{L^2}\sum_{ij} e^{-i Q \cdot (r_i - r_j)} <s_i s_j>
\end{equation}
$$

In [ ]:
include("obser.jl")
include("core.jl")

## Example Run

Define the `main` funciton.

In [ ]:
function main()
	hm = Hubbard_Model
	p = Hubbard_Model.Param

	aux_field = Hubbard_Model.gen_auxf()
	nlist = Hubbard_Model.gen_nlist()
	T = Hubbard_Model.T(nlist)

	# This could be slow, since no checkerboard decomposition.
	exp_T = exp(p.Δτ * T)
	exp_mT = exp(-p.Δτ * T)

	sl = Hubbard_Model.Square_Lattice(aux_field, nlist, T, exp_T, exp_mT)

	B_up_l, B_dn_l = CoreM.init_B_mat_list(sl.aux_field, sl.exp_mT, p.MatDim, p.N_time_slice)

	B_τ_0_up = CoreM.B_τ_0(p.N_time_slice, B_up_l, p.MatDim)
	B_τ_0_dn = CoreM.B_τ_0(p.N_time_slice, B_dn_l, p.MatDim)

	B_β_τ_up = CoreM.B_β_τ(0, B_up_l, p.MatDim, p.N_time_slice)
	B_β_τ_dn = CoreM.B_β_τ(0, B_dn_l, p.MatDim, p.N_time_slice)

	G_up = CoreM.G_σ_τ_τ_calc(B_τ_0_up, B_β_τ_up, p.MatDim)
	G_dn = CoreM.G_σ_τ_τ_calc(B_τ_0_dn, B_β_τ_dn, p.MatDim)

	println("initialization done")

	for i=1:p.N_warmup
		print(i,"...")
		CoreM.sweep!(G_up, G_dn, B_up_l, B_dn_l, sl, p.MatDim, p.N_ns_int, p.N_time_slice, false)
	end
	println()
	for i=1:p.N_bin
		println("bin=",i)
		for j=1:p.N_sweep
			print(j,"...")
			CoreM.sweep!(G_up,G_dn, B_up_l, B_dn_l, sl, p.MatDim, p.N_ns_int, p.N_time_slice, true)
		end
		Obser.bin_store(i)
		println()
	end
end

Run `main` function, this is where the program acctually start.

In [ ]:
main()